# Obtaining the best redshift from 3 Catalogs: Griffiths, 3dHST, UltraVISTA

## What we have:
## Griffiths has spec-z, photo-z. Their best Z is 'high-quality spec z else photoz.' It is not known how high-quality is defined. The errors listed on their photoz are 3sig. 
## 3dHST has spec-zs, photo-zs, and grism-zs. Their 'z_best' can be any of these.
## UltraVISTA just has photo-zs. 
## MUSYC has spec-zs and photo-z's. 

## Workflow for choosing best redshift:
## 1) If Griffiths has high quality spec z (z_spec = z_best), use that.  Note: Griffith et al states that SPECZ = Z if z_quality >=3, but that doens't seem to be the case for places that don't have a quality measurement - inspection showed that in these cases, the specz was more consistent with the other catalogs than the photoz, so I'm using z_spec = z_best as the critera here instead of z_qual >=3. 
## else if 3dHST spec-z = 3dHST best_z, use that. 
## else if MUSYC has spec-z (within real range; 0.01< z < 5), use that. 
## else (no good spec-z's anywhere):
## compare 1-sig errors on Griffiths photoz, 3dHST grism z, 3dHST photoz, UltraVISTA photoz MUSYC. choose that with lowest err. Note: all except Griffiths report 1 sig errors; Griffiths reports 3sig if Imaging = COSMOS, so that error is divided by 3 in the comparison.

 



In [1]:
%matplotlib inline

from astropy.io import fits as pyfits
from astropy.table import Table
import numpy as np
from astropy.io.fits import Column
from datetime import datetime
from operator import itemgetter
import matplotlib.pyplot as plt
import os
import warnings
import requests

In [2]:
def download_from_dropbox(url):
    
    local_filename = "{:}".format(url.split("/")[-1].split("?")[0])
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                f.flush()
            
    return local_filename

In [3]:
old_catalog_filename = download_from_dropbox("https://www.dropbox.com/s/ge7sgtf0crxmwyi/all_redshift_info.fits?dl=1")
old_catalog = Table.read(old_catalog_filename)


/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:315: SNIMissingWarning: An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#snimissingwarning.
  SNIMissingWarning
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:120: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:120: InsecurePlatformWarning: A true SSLContext object is not available. This prevent

In [4]:
subjects=set(old_catalog['OBJNO'])
imagings=set(old_catalog['IMAGING'])

In [67]:
#add new redshift columns 
col1=Table.Column(name='Z_BEST',length=len(old_catalog))
col2=Table.Column(name='Z_BEST_TYPE',length=len(old_catalog),dtype='|S10')
col3=Table.Column(name='Z_BEST_SOURCE',length=len(old_catalog),dtype='|S10')

In [68]:
updated_table = old_catalog.copy(copy_data=True)
updated_table.add_columns((col1,col2,col3))

In [128]:
def get_best_redshift(galaxy):
    
    #define Griffith variables
    griffith_specz = galaxy['SPECZ']
    griffith_quality = galaxy['ZQUALITY']
    griffith_photoz = galaxy['PHOTOZ']
    griffith_bestz = galaxy['Z']
    
    if galaxy['IMAGING']=='COSMOS    ':
        griffith_photoz_err = galaxy['PHOTOZ_ERR']/galaxy['PHOTOZ']/3.
    else:
        griffith_photoz_err = galaxy['PHOTOZ_ERR']/galaxy['PHOTOZ']
    
    if str(griffith_photoz_err)=='nan':
        griffith_photoz_err = 99
    if galaxy['PHOTOZ_ERR']<=0:
        griffith_photoz_err = 99

    #define 3dhst variables
    dhst_specz = galaxy['z_spec']
    dhst_bestz = galaxy['z_best']
    dhst_photoz = galaxy['z_peak_phot']
    dhst_photoz_err = (galaxy['z_phot_u68']-galaxy['z_phot_l68'])/dhst_photoz

    dhst_grismz = galaxy['z_max_grism']
    dhst_grismz_err = (galaxy['z_grism_u68']-galaxy['z_grism_l68'])/dhst_grismz

    
    if str(dhst_photoz_err)=='nan':
        dhst_photoz_err = 99
    if dhst_photoz < 0:
        dhst_photoz_err = 99
    if str(dhst_grismz_err)=='nan':
        dhst_grismz_err = 99
    if dhst_grismz < 0:
        dhst_grismz_err = 99
    
    #define ultravista variables
    uv_photoz = galaxy['zPDF']
    uv_photoz_err = (galaxy['zPDF_u68']-galaxy['zPDF_l68'])/uv_photoz
    
    if str(uv_photoz_err)=='nan':
        uv_photoz_err = 99
    if uv_photoz < 0:
        uv_photoz_err=99
        
    #define MUSYC variables:
    musyc_specz = galaxy['Z_SPEC_ALL']
    musyc_photoz = galaxy['Z_PEAK']
    musyc_photoz_err = (galaxy['z_U68']-galaxy['z_L68'])/musyc_photoz
    
    if str(musyc_photoz_err)=='nan':
        musyc_photoz_err = 99
    if musyc_photoz < 0:
        musyc_photoz_err=99
    
    #Order of operations to determine which redshift to use:
    #1) If high quality specz exists in Griffiths catalog, use that first. 
    if griffith_specz==griffith_bestz and griffith_specz > 0 and griffith_specz < 9:
        return griffith_specz,'SPEC_Z','Griffith'
        
    #if no specz in Griffith, check 3Dhst
    elif dhst_specz == dhst_bestz and dhst_specz >0 and dhst_specz <9: 
            return dhst_bestz,'SPEC_Z','3DHST'
    #if no specz in Griffith or 3Dhst, check MUSYC:
    elif musyc_specz > 0 and musyc_specz < 9:
            return musyc_specz, 'SPEC_Z','MUSYC'
        
    else: #no speczs, so check which photoz or grism has smallest error. 
        photoz_list = [[griffith_photoz,griffith_photoz_err,'PHOTO_Z','Griffith'],[dhst_photoz,dhst_photoz_err,'PHOTO_Z','3DHST'],[uv_photoz,uv_photoz_err,'PHOTO_Z','UltraVISTA'],[dhst_grismz,dhst_grismz_err,'GRISM_Z','3DHST'],[musyc_photoz,musyc_photoz_err,'PHOTO_Z','MUSYC']]
        photoz_list = sorted(photoz_list, key=itemgetter(1)) #sort by which has smallest error            
        # if photo_z is a real number: 
        if photoz_list[0][0] > 0 and photoz_list[0][0] <9:
            return photoz_list[0][0],photoz_list[0][2],photoz_list[0][3]
        #else no good redshifts anywhere, return blank
        else:
            return 0,'',''

In [129]:
for g,galaxy in enumerate(updated_table):
    galaxy['Z_BEST'], galaxy['Z_BEST_TYPE'], galaxy['Z_BEST_SOURCE'] = get_best_redshift(galaxy)
    if g % 10000==0:
        print g 

0
10000
20000
30000
40000
50000
60000
70000
80000

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:10: RuntimeWarning: invalid value encountered in float_scalars
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:10: RuntimeWarning: divide by zero encountered in float_scalars



90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000


In [130]:
updated_table.write('/home/mel/Dropbox/gzhubble/hubble_files/gzh_redshift_data/best_redshift_table_%i_%i_%i.fits'%(datetime.now().month,datetime.now().day,datetime.now().year),overwrite=True)


In [144]:
#Check distribution of different redshifts
imaging_list=list(set(updated_table['IMAGING']))

z_sources = list(set(updated_table['Z_BEST_SOURCE']))
z_types = list(set(updated_table['Z_BEST_TYPE']))
for survey in imaging_list:
    print('for survey %s:' %survey)
    for source in z_sources: 
        print('for source %s'%source)
        for zt in z_types:
            print ('%s:  = %i' %(zt, ((updated_table['IMAGING'] == survey) & (updated_table['Z_BEST_SOURCE'] == source)&(updated_table['Z_BEST_TYPE'] == zt)).sum()))
    print('')
for source in z_sources:
    print('total %s = %i'%(source,((updated_table['Z_BEST_SOURCE']==source)).sum()))
    
    print('')
for zt in z_types:
    print('total %s = %i'%(zt,((updated_table['Z_BEST_TYPE']==zt)).sum()))


for survey AEGIS     :
for source 
GRISM_Z:  = 0
SPEC_Z:  = 0
PHOTO_Z:  = 0
:  = 1134
for source 3DHST
GRISM_Z:  = 515
SPEC_Z:  = 12
PHOTO_Z:  = 249
:  = 0
for source UltraVISTA
GRISM_Z:  = 0
SPEC_Z:  = 0
PHOTO_Z:  = 0
:  = 0
for source MUSYC
GRISM_Z:  = 0
SPEC_Z:  = 0
PHOTO_Z:  = 0
:  = 0
for source Griffith
GRISM_Z:  = 0
SPEC_Z:  = 3656
PHOTO_Z:  = 2941
:  = 0

for survey GOODS-N   :
for source 
GRISM_Z:  = 0
SPEC_Z:  = 0
PHOTO_Z:  = 0
:  = 159
for source 3DHST
GRISM_Z:  = 101
SPEC_Z:  = 5
PHOTO_Z:  = 253
:  = 0
for source UltraVISTA
GRISM_Z:  = 0
SPEC_Z:  = 0
PHOTO_Z:  = 0
:  = 0
for source MUSYC
GRISM_Z:  = 0
SPEC_Z:  = 0
PHOTO_Z:  = 0
:  = 0
for source Griffith
GRISM_Z:  = 0
SPEC_Z:  = 1997
PHOTO_Z:  = 36
:  = 0

for survey GOODS_FULL:
for source 
GRISM_Z:  = 0
SPEC_Z:  = 0
PHOTO_Z:  = 0
:  = 11946
for source 3DHST
GRISM_Z:  = 2893
SPEC_Z:  = 745
PHOTO_Z:  = 1662
:  = 0
for source UltraVISTA
GRISM_Z:  = 0
SPEC_Z:  = 0
PHOTO_Z:  = 0
:  = 0
for source MUSYC
GRISM_Z:  = 0
SPEC_Z:  = 

In [145]:
len(updated_table)

192258